# Maximum Likelihood Estimation (MLE)

- 관측 데이터가 가장 그럴듯하게 나올 수 있는 모델 파라미터를 찾는 방법
- 특정 데이터 x에 대해 파라미터 θ 하에서 데이터가 나올 확률이 나올 때, 이를 최대화시키는 θ를 찾는 것

θ_MLE = argmax L(θ | x)
- L(θ | x) : Likelihood Function으로, 파라미터 θ 하에서 x가 나올 확률
- θ_MLE : MLE로 추정한 최적 파라미터

베르누이 분포를 이용해 예시를 들어보자. 

압정을 100번 던져서 머리와 다리 중, 머리로 떨어진 횟수가 27번이다. 
이 때 L(θ | x) = L(p) = p ^ 27 * (1 - p) ^ (100 - 27) 으로 나온다. 

p에 따른 L(p)를 최대화 하기 위한 θ는 어떻게 구할까? 

1. 로그를 씌운다. 
    logL(p) = 27logp + 73log(1-p)

2. 미분을 하고 그 값을 0으로 놓는다(미분시 0은 최대, 최소값을 의미함)
    27 / p - 73 / (1-p) = 0, 27/p = 73/(1-p)

3. p를 구한다. 
    73p = 27 - 27p, p = 0.27

이번 경우에서 최적의 파라미터는 0.27이 나온다. 

MLE는 확률 모델을 정하고, L(p)를 정의한 후 Likelihood를 최대화하는 파라미터를 찾는 방법이다

# Overfitting(과적합)

과적합시, 테스트 데이터에 대해서는 원하는 결과를 얻을 수 있지만, 다른 데이터에서는 정확한 답이 나오지 않을 수 있다. 

그렇다면 어떻게 과적합을 예방할 수 있을까? 
- 데이터 크기 증가
- feature를 줄이기
- Regularization를 이용해 과적합 방지
    - Early Stopping : 중간에 validation loss가 낮아지지 않을 때 멈추는 것
    - Reducing Network Size : 뉴럴 네트워크의 사이즈를 줄이는 것
    - Weight Decay : W 파라미터의 크기 제한
    - Droptout : 딥러닝시 뉴런을 일부 랜덤으로 꺼버리는 방식(뉴런 의존성을 줄일 수 있다)
    - Batch Normalization : 각 레이어의 출력을 평균 0, 분산을 1로 맞추는 방식


# Basic Approach to train DNN

1. 뉴럴 네트워크를 설계한다. ex) 입력 데이터가 vector이며 10개의 feature가 있다면, 10차원의 입력을 받아 이를 토대로 5가지의 클래스로 만들어야 한다면 softmax 10 -> 5가 나온다. 
2. 훈련 및 과적합 체크
    - 만약 과적합이 아니라면 모델 사이즈를 증가시킨다. 
    - 과적합이 발생한다면 Regularization을 추가한다. 
3. 2번을 반복한다. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x_train = torch.FloatTensor([[1, 2, 1], 
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 2],
                             [1, 6, 6],
                             [1, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [3]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 3])
torch.Size([8])


shape을 통해서, 우리가 해야하는 것이 classification이라는 것을 알 수 있다. 

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [5]:
class SoftmaxClassifierModel(nn.Module): 
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    
    def forward(self, x):
        return self.linear(x)

In [6]:
model = SoftmaxClassifierModel()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [8]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    
    for epoch in range(nb_epochs):
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [9]:
# Test (validation)

def test (model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

In [10]:
train(model, optimizer, x_train, y_train)

Epoch    0 / 20 Cost: 2.272475
Epoch    1 / 20 Cost: 1.250806
Epoch    2 / 20 Cost: 1.199265
Epoch    3 / 20 Cost: 1.174748
Epoch    4 / 20 Cost: 1.158751
Epoch    5 / 20 Cost: 1.146726
Epoch    6 / 20 Cost: 1.136171
Epoch    7 / 20 Cost: 1.126433
Epoch    8 / 20 Cost: 1.117174
Epoch    9 / 20 Cost: 1.108253
Epoch   10 / 20 Cost: 1.099598
Epoch   11 / 20 Cost: 1.091173
Epoch   12 / 20 Cost: 1.082957
Epoch   13 / 20 Cost: 1.074937
Epoch   14 / 20 Cost: 1.067105
Epoch   15 / 20 Cost: 1.059454
Epoch   16 / 20 Cost: 1.051978
Epoch   17 / 20 Cost: 1.044672
Epoch   18 / 20 Cost: 1.037530
Epoch   19 / 20 Cost: 1.030549


파라미터가 optimizer를 통해 MLE를 계산하며 최적값을 구해지고 있다. 

In [11]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.518564


LOSS값이 늘어난 상태이다. 

## Learning Rate

lr이 너무 크면 diverge하면서 cost가 점점 증가한다. 
lr이 너무 작으면 cost가 거의 줄어들지 않는다. 

In [12]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train(model, optimizer, x_train, y_train)

Epoch    0 / 20 Cost: 1.312794
Epoch    1 / 20 Cost: 1.011654
Epoch    2 / 20 Cost: 1.005033
Epoch    3 / 20 Cost: 0.998682
Epoch    4 / 20 Cost: 0.992507
Epoch    5 / 20 Cost: 0.986491
Epoch    6 / 20 Cost: 0.980623
Epoch    7 / 20 Cost: 0.974895
Epoch    8 / 20 Cost: 0.969301
Epoch    9 / 20 Cost: 0.963836
Epoch   10 / 20 Cost: 0.958496
Epoch   11 / 20 Cost: 0.953276
Epoch   12 / 20 Cost: 0.948173
Epoch   13 / 20 Cost: 0.943184
Epoch   14 / 20 Cost: 0.938304
Epoch   15 / 20 Cost: 0.933531
Epoch   16 / 20 Cost: 0.928862
Epoch   17 / 20 Cost: 0.924293
Epoch   18 / 20 Cost: 0.919822
Epoch   19 / 20 Cost: 0.915446


In [13]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0% Cost: 0.428293


# Data Preprocessing(데이터 전처리)



In [14]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

값이 특정 classes로 만들어지는 것은 아니다. 
regression을 시도해볼 수 있다. 
그렇다면 mean-square-error를 이용할 수 있다. 

이 때 데이터 전처리를 하면 계산이 수월해진다. 

In [15]:
mu = x_train.mean(dim=0)

In [16]:
sigma = x_train.std(dim=0)
print(mu)
print(sigma)

tensor([84.8000, 84.6000, 85.6000])
tensor([11.0544, 12.2393, 12.6214])


In [17]:
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Standardazation(정규화)를 이용해 데이터를 전처리해줄 수 있다. 
위의 데이터가 정규분포를 따른다고 가정하고 mu, sigma를 이용해 전처리를 진행해준다. 